### clone代码

In [ ]:
!git clone https://github.com/modelscope/modelscope-agent.git

### 安装特定依赖

In [ ]:
!cd modelscope-agent && pip install -r requirements.txt
!pip install vllm

### 本地配置

In [ ]:
import os
os.chdir('modelscope-agent/demo')

import sys
sys.path.append('../')

### 本地部署llm
modelscope提供模型本地启动服务功能。这里我们使用该功能，将模型部署成openai api兼容的接口。具体操作可参考如下：
#### 下载模型

In [ ]:
!python -c "from modelscope import snapshot_download; model_dir = snapshot_download('qwen/Qwen1.5-7B-Chat', cache_dir='qwen1.5-7b-chat');print(model_dir)"

#### 部署模型

In [ ]:
# 在终端里执行下面的命令启动llm模型服务
!MODELSCOPE_CACHE='qwen1.5-7b-chat' python -m vllm.entrypoints.openai.api_server \
    --model qwen/Qwen1.5-7B-Chat --dtype=half --max-model-len 8192  --gpu-memory-utilization 0.95

#### 测试部署
测试模型服务，如果正确返回，说明模型服务部署完成。

In [ ]:
# 在终端里执行下面的命令，测试模型服务
!curl http://localhost:8000/v1/chat/completions \
    -H "Content-Type: application/json" \
    -d '{
        "model": "qwen/Qwen1.5-7B-Chat",
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "写一篇春天为主题的作文"}
        ],
        "stop": ["<|im_end|>", "<|endoftext|>"]
    }'

### 构建agent

In [ ]:
from modelscope_agent.agents import RolePlay

role_template = '你是一个agent小助手，你需要根据用户的要求来回答他们的问题'
llm_config = {
    'model': 'qwen/Qwen1.5-7B-Chat', 
    'model_server': 'openai',
    'api_base':'http://127.0.0.1:8000/v1',
    'api_key': 'EMPTY'
    }
function_list = []

bot = RolePlay(function_list=function_list,llm=llm_config, instruction=role_template)

### 执行agent

In [ ]:
response = bot.run("你好，请以李云龙的语气和我对话")
text = ''
for chunk in response:
    text += chunk
print(text)

agentfabric也支持本地部署llm，若您需要使用，可以查看文档：[docs/local_deploy.md](../docs/local_deploy.md)